In [1]:
%matplotlib inline
from copy import deepcopy
from collections import OrderedDict
import gc
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD,Adam,lr_scheduler
from torch.utils.data import random_split
import torchvision
from torchvision import transforms,models

In [2]:
transform = transforms.Compose([
    transforms.Resize(256), #Resizing all the images
    transforms.CenterCrop(224),#Cropping all from centre
    transforms.ToTensor(),#Converting to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),#Normalize all using the mean and standard deviation (x-mu/rho)
]) #Data preparation
traindata = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainset,valset = random_split(traindata,[42000,8000])
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=64,shuffle=False)

test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(test_data, batch_size=4, shuffle=False, num_workers=2)  #same for the testing data

classes = ('Airplane', 'Car', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck')

100%|██████████| 170498071/170498071 [00:02<00:00, 70541236.25it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        base = models.resnet18(pretrained=True)
        self.base = nn.Sequential(*list(base.children())[:-1])
        in_features = base.fc.in_features
        self.drop = nn.Dropout()
        self.final = nn.Linear(in_features,10)

    def forward(self,x):
        x = self.base(x)
        x = self.drop(x.view(-1,self.final.in_features))
        return self.final(x)

model = Model().cuda()
[x for x,y in model.named_children()]

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 147MB/s]


['base', 'drop', 'final']

In [4]:
criterion = nn.CrossEntropyLoss()
param_groups = [
    {'params':model.base.parameters(),'lr':.0001},
    {'params':model.final.parameters(),'lr':.001}
]
optimizer = Adam(param_groups)
lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)
states = {}

In [5]:
%%time
best_val_acc = -1000
best_val_model = None
for epoch in range(5):
    model.train(True)
    running_loss = 0.0
    running_acc = 0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.cuda(),labels.cuda()

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item() * inputs.size(0)
        out = torch.argmax(outputs.detach(),dim=1)
        assert out.shape==labels.shape
        running_acc += (labels==out).sum().item()
    print(f"Train loss {epoch+1}: {running_loss/len(trainset)},Train Acc:{running_acc*100/len(trainset)}%")

    correct = 0
    model.train(False)
    with torch.no_grad():
        for inputs,labels in valloader:

          out = model(inputs.cuda()).cpu()
          out = torch.argmax(out,dim=1)
          acc = (out==labels).sum().item()
          correct += acc
    print(f"Val accuracy:{correct*100/len(valset)}%")
    if correct>best_val_acc:

      best_val_acc = correct
      best_val_model = deepcopy(model.state_dict())
    lr_scheduler.step()

print('Finished Training')

Train loss 1: 0.43862458271072025,Train Acc:85.26428571428572%
Val accuracy:92.1625%
Train loss 2: 0.12434501542080016,Train Acc:96.13095238095238%
Val accuracy:94.25%
Train loss 3: 0.08531353051747595,Train Acc:97.41904761904762%
Val accuracy:94.05%
Train loss 4: 0.08253810505781854,Train Acc:97.58809523809524%
Val accuracy:94.1875%
Train loss 5: 0.08204647142688433,Train Acc:97.60714285714286%
Val accuracy:94.0125%
Finished Training
CPU times: user 17min 1s, sys: 1min 36s, total: 18min 38s
Wall time: 18min 29s


In [7]:
%%time
correct = 0
model.load_state_dict(best_val_model)
model.train(False)
with torch.no_grad():
    for inputs,labels in testloader:
        out = model(inputs.cuda()).cpu()
        out = torch.argmax(out,dim=1)
        acc = (out==labels).sum().item()

        correct += acc
print(f"Test accuracy: {correct*100/len(test_data)}%")

Test accuracy: 94.06%
CPU times: user 15.9 s, sys: 2.67 s, total: 18.6 s
Wall time: 28 s
